<h1 align="center">LAB1S1_p2. PoS Tagging with HMM </h1>

<h3 style="display:block; margin-top:5px;" align="center">Natural Language and Information Retrieval</h3>
<h3 style="display:block; margin-top:5px;" align="center">Degree in Data Science</h3>
<h3 style="display:block; margin-top:5px;" align="center">2024-2025</h3>    
<h3 style="display:block; margin-top:5px;" align="center">ETSInf. Universitat Politècnica de València</h3>
<br>

### Put your names here

- Kacper Multan
- Filip Polacik

## **Installing necessary packages using pip**

In [32]:
!pip install numpy
!pip install nltk
!pip install scikit-learn

/usr/bin/pip:6: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  from pkg_resources import load_entry_point
/usr/bin/pip:6: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  from pkg_resources import load_entry_point
/usr/bin/pip:6: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  from pkg_resources import load_entry_point


## **Downloading  the NLTK resources using the download method**

In [33]:
import nltk

# brown corpus
nltk.download('brown')

# punkt tokenizer
nltk.download('punkt_tab')
# or
# nltk.download('punkt')
# it depends on your nltk version

# stopwords list
nltk.download('stopwords')

[nltk_data] Downloading package brown to /home/zzzdream/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/zzzdream/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/zzzdream/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## **Accessing a specific corpus (Brown corpus)**

In [34]:
from nltk.corpus import brown

print(f"{len(brown.sents())} sentences in the corpus.")
print("First sentence:")
brown.sents()[0]

57340 sentences in the corpus.
First sentence:


['The',
 'Fulton',
 'County',
 'Grand',
 'Jury',
 'said',
 'Friday',
 'an',
 'investigation',
 'of',
 "Atlanta's",
 'recent',
 'primary',
 'election',
 'produced',
 '``',
 'no',
 'evidence',
 "''",
 'that',
 'any',
 'irregularities',
 'took',
 'place',
 '.']

In [35]:
nwords = 0 # number of words
voc, dtags = set(), set() # vocabulary, different tags

for s in brown.tagged_sents():
    #print(s)
    ws = [w for w, _ in s]
    ts = [t for _, t in s]
    nwords += len(ws)
    voc.update(ws)
    dtags.update(ts)

print(f"Number of words: {nwords}")
print(f"Vocabulary size: {len(voc)}")
print(f"Number of different tags: {len(dtags)}")
print("First sentence tagged:")
brown.tagged_sents()[0]

Number of words: 1161192
Vocabulary size: 56057
Number of different tags: 472
First sentence tagged:


[('The', 'AT'),
 ('Fulton', 'NP-TL'),
 ('County', 'NN-TL'),
 ('Grand', 'JJ-TL'),
 ('Jury', 'NN-TL'),
 ('said', 'VBD'),
 ('Friday', 'NR'),
 ('an', 'AT'),
 ('investigation', 'NN'),
 ('of', 'IN'),
 ("Atlanta's", 'NP$'),
 ('recent', 'JJ'),
 ('primary', 'NN'),
 ('election', 'NN'),
 ('produced', 'VBD'),
 ('``', '``'),
 ('no', 'AT'),
 ('evidence', 'NN'),
 ("''", "''"),
 ('that', 'CS'),
 ('any', 'DTI'),
 ('irregularities', 'NNS'),
 ('took', 'VBD'),
 ('place', 'NN'),
 ('.', '.')]

### Brown corpus Tagset

- https://varieng.helsinki.fi/CoRD/corpora/BROWN/tags.html
- http://korpus.uib.no/icame/manuals/BROWN/INDEX.HTM

In [36]:
print("raw corpus:")
print(brown.raw()[:500])

raw corpus:


	The/at Fulton/np-tl County/nn-tl Grand/jj-tl Jury/nn-tl said/vbd Friday/nr an/at investigation/nn of/in Atlanta's/np$ recent/jj primary/nn election/nn produced/vbd ``/`` no/at evidence/nn ''/'' that/cs any/dti irregularities/nns took/vbd place/nn ./.


	The/at jury/nn further/rbr said/vbd in/in term-end/nn presentments/nns that/cs the/at City/nn-tl Executive/jj-tl Committee/nn-tl ,/, which/wdt had/hvd over-all/jj charge/nn of/in the/at election/nn ,/, ``/`` deserves/vbz the/at praise/nn and/c


## **Tokenizing sentences and words with NLTK**

In [37]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize

# Text from "https://www.nytimes.com/2024/02/20/world/europe/interpol-strongmen-abuse.html"

text="""For years, strongmen and autocrats had a novel weapon in their hunt for political enemies.
They used Interpol, the world’s largest police organization, to reach across borders and grab them — even in democracies.
An award-winning Venezuelan journalist was detained in Peru. An Egyptian asylum seeker was stopped in Australia.
And Russia has tried repeatedly to secure the arrest of William F. Browder, a London-based human rights campaigner."""

sents=sent_tokenize(text)
sents

['For years, strongmen and autocrats had a novel weapon in their hunt for political enemies.',
 'They used Interpol, the world’s largest police organization, to reach across borders and grab them — even in democracies.',
 'An award-winning Venezuelan journalist was detained in Peru.',
 'An Egyptian asylum seeker was stopped in Australia.',
 'And Russia has tried repeatedly to secure the arrest of William F. Browder, a London-based human rights campaigner.']

In [38]:
sent_tokens = []
for s in sents:
    aux = word_tokenize(s)
    sent_tokens.append(aux)
    print(aux)
print(sent_tokens)

['For', 'years', ',', 'strongmen', 'and', 'autocrats', 'had', 'a', 'novel', 'weapon', 'in', 'their', 'hunt', 'for', 'political', 'enemies', '.']
['They', 'used', 'Interpol', ',', 'the', 'world', '’', 's', 'largest', 'police', 'organization', ',', 'to', 'reach', 'across', 'borders', 'and', 'grab', 'them', '—', 'even', 'in', 'democracies', '.']
['An', 'award-winning', 'Venezuelan', 'journalist', 'was', 'detained', 'in', 'Peru', '.']
['An', 'Egyptian', 'asylum', 'seeker', 'was', 'stopped', 'in', 'Australia', '.']
['And', 'Russia', 'has', 'tried', 'repeatedly', 'to', 'secure', 'the', 'arrest', 'of', 'William', 'F.', 'Browder', ',', 'a', 'London-based', 'human', 'rights', 'campaigner', '.']
[['For', 'years', ',', 'strongmen', 'and', 'autocrats', 'had', 'a', 'novel', 'weapon', 'in', 'their', 'hunt', 'for', 'political', 'enemies', '.'], ['They', 'used', 'Interpol', ',', 'the', 'world', '’', 's', 'largest', 'police', 'organization', ',', 'to', 'reach', 'across', 'borders', 'and', 'grab', 'them

## Removing the stopwords in sentences

In [39]:
from nltk.corpus import stopwords
stopw = set(stopwords.words("english"))

for sent in sent_tokens:
    print([x for x in sent if x.lower() not in stopw])

['years', ',', 'strongmen', 'autocrats', 'novel', 'weapon', 'hunt', 'political', 'enemies', '.']
['used', 'Interpol', ',', 'world', '’', 'largest', 'police', 'organization', ',', 'reach', 'across', 'borders', 'grab', '—', 'even', 'democracies', '.']
['award-winning', 'Venezuelan', 'journalist', 'detained', 'Peru', '.']
['Egyptian', 'asylum', 'seeker', 'stopped', 'Australia', '.']
['Russia', 'tried', 'repeatedly', 'secure', 'arrest', 'William', 'F.', 'Browder', ',', 'London-based', 'human', 'rights', 'campaigner', '.']


# **PoS tagging with HMM using NLTK**

Using the [**HiddenMarkovModelTagger**](https://tedboy.github.io/nlps/generated/generated/nltk.tag.HiddenMarkovModelTagger.html) Object

## Training a HMM tagger using Brown corpus

In [40]:
import nltk.tag as tagger

# create training and test partitions
train_corpus = brown.tagged_sents()[:2000]
test_corpus = brown.tagged_sents()[2000:2500]

# train the model
mytagger=tagger.HiddenMarkovModelTagger.train(train_corpus)

# evaluate the model behavior
acc = mytagger.accuracy(test_corpus)
print(f"Accuracy: {acc:0.5}")

Accuracy: 0.80898


## Tagging a sentence

In [41]:
s = "The young boy can open the can with his hands."
ws = word_tokenize(s)

print(ws)
print(mytagger.tag(ws)) # This method labels a word sequence.

['The', 'young', 'boy', 'can', 'open', 'the', 'can', 'with', 'his', 'hands', '.']
[('The', 'AT'), ('young', 'JJ'), ('boy', 'NN'), ('can', 'MD'), ('open', 'VB'), ('the', 'AT'), ('can', 'NN'), ('with', 'IN'), ('his', 'PP$'), ('hands', 'NNS'), ('.', '.')]


# Simplifying the Brown corpus

Apply the following criterion:

- All tags will be simplified to keep only the morphological information.
- Replace the prefix “FW-” and split the PoS tag by hyphens and keep the PoS
at the first position.

Notice a specific tag “--” denoting a dash symbol.

In [42]:
def simplify_tag(tag):
    fst, snd = tag
    simplified_snd = snd.split("-")[0]
    return fst, simplified_snd

def simplify_corpus(corpus, rule):
    simplifiedCorpus = []
    for sentence in corpus:
        simplifiedSentence = []
        for tag in sentence:
            simplifiedSentence.append(rule(tag))

        simplifiedCorpus.append(simplifiedSentence)

    return simplifiedCorpus


simplifiedBrown = simplify_corpus(brown.tagged_sents(), simplify_tag)

nwords = 0 # number of words
voc, dtags = set(), set() # vocabulary, different tags
for s in simplifiedBrown:
    #print(s)
    ws = [w for w, _ in s]
    ts = [t for _, t in s]
    nwords += len(ws)
    voc.update(ws)
    dtags.update(ts)

print(f"Number of words: {nwords}")
print(f"Vocabulary size: {len(voc)}")
print(f"Number of different tags: {len(dtags)}")
print("First sentence with simplified tags:")
simplifiedBrown[0]

Number of words: 1161192
Vocabulary size: 56057
Number of different tags: 181
First sentence with simplified tags:


[('The', 'AT'),
 ('Fulton', 'NP'),
 ('County', 'NN'),
 ('Grand', 'JJ'),
 ('Jury', 'NN'),
 ('said', 'VBD'),
 ('Friday', 'NR'),
 ('an', 'AT'),
 ('investigation', 'NN'),
 ('of', 'IN'),
 ("Atlanta's", 'NP$'),
 ('recent', 'JJ'),
 ('primary', 'NN'),
 ('election', 'NN'),
 ('produced', 'VBD'),
 ('``', '``'),
 ('no', 'AT'),
 ('evidence', 'NN'),
 ("''", "''"),
 ('that', 'CS'),
 ('any', 'DTI'),
 ('irregularities', 'NNS'),
 ('took', 'VBD'),
 ('place', 'NN'),
 ('.', '.')]

## **Splitting a corpus in TRAIN and TEST partitions using SKLEARN**

In [43]:
from sklearn.model_selection import train_test_split

# Notice that, simplifiedBrown is the transformed corpus with the simplified labels.
train_part, test_part = train_test_split(simplifiedBrown, random_state=1234, test_size=0.1, shuffle=True)

# Training the corpus using the train partition

In [44]:
###### COMPLETE #######
newtagger = tagger.HiddenMarkovModelTagger.train(train_corpus)

## Evaluating the model using the test partition

In [45]:
# This method evaluates a tagged-sequence in terms of accuracy
# How the predicted labels match the ground truth labels in the corpus
acc = newtagger.accuracy(test_part)
print(f"Accuracy: {acc:0.5}")

# This method returns the confusion matrix
# but you cannot see anything in the notebook
print(newtagger.confusion(test_part))

Accuracy: 0.79212
           |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           W                                                             |
           |                   